In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import random
import os
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
file_no = 9
df_collapsed    = pl.read_parquet(f'../../data/stanford/facebook/348.edges_collapsed_edges.{file_no}.parquet')
df_edge_arc_pos = pl.read_parquet(f'../../data/stanford/facebook/348.edges_arc_pos.{file_no}.parquet')
df_collapsed.sample(10)

In [ ]:
df_edge_arc_pos.sample(1)

In [4]:
def extractCircles(_df_):
    circles, circle_lu = [], {}
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        circle_lu[k] = len(circles)
        circles.append(k)
    return circles, circle_lu
def validatePositions(_df_, clearance=10): # edge_arc_pos
    # Extract the circles first
    circles, circle_lu = extractCircles(_df_)
    # Then make sure the positions don't fall within a circle & that they have some clearance
    for k, k_df in _df_.group_by(['x','y', 'x_circle', 'y_circle', 'r_circle']):
        _xy_     = (k[0], k[1])
        _uv_     = rt.unitVector((k[2:4], _xy_))
        _xy_end_ = (_xy_[0]+ clearance*_uv_[0], _xy_[1]+ clearance*_uv_[1])
        _my_circle_ = k[2:5]
        for c in circles:
            if c == _my_circle_: continue
            _length_ = rt.segmentLength((_xy_,c))
            if _length_ < c[2] + clearance:
                raise Exception(f'node key "{k_df["node_key"]}" with radius circle {c}')
            _length_ = rt.segmentLength((_xy_end_, c))

def extents(_df_): # edge_arc_pos -- assumes that the coordinates are already in screen space
    xmax, ymax = 10.0, 10.0
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        xmax, ymax = max(xmax, k[0]+k[2]), max(ymax, k[1]+k[2])
    for k, k_df in _df_.group_by(['x','y']):
        xmax, ymax = max(xmax, k[0]), max(ymax, k[1])
    return (0.0, 0.0, xmax, ymax)

validatePositions(df_edge_arc_pos)
_ext_ = extents(df_edge_arc_pos)

In [5]:
x_ins, y_ins = 50, 50
# Base Circles
svg_base = [f'<svg x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}">']
svg_base.append(f'<rect x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}" fill="#ffffff" />')
circles, circles_lu = extractCircles(df_edge_arc_pos)
for i in range(len(circles)):
    cx, cy, r = circles[i]
    svg_base.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3" />')
# Entry / Exit Points
svg_pts = []
for k, k_df in df_edge_arc_pos.group_by(['node_key', 'x', 'y', 'x_circle', 'y_circle', 'r_circle']):
    if '_to_' in k[0]: _color_, _fill_ = '#ff0000', 'none'
    else:              _color_, _fill_ = '#00ff00', '#00b000'
    _xy_, _cxy_ = k[1:3], k[3:5]
    svg_pts.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="4" fill="{_fill_}" stroke="{_color_}" stroke-width="0.5"/>')
    _uv_        = rt.unitVector((_cxy_, _xy_))
    svg_pts.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy_[0]+10*_uv_[0]}" y2="{_xy_[1]+10*_uv_[1]}" stroke="#404040" stroke-width="1.0"/>')
# Delauney Triangulations
_box_ = [(_ext_[0],_ext_[1]),(_ext_[0],_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[1])]
voronoi_polys       = rt.isedgarVoronoi(circles, Box=_box_, use_circle_radius=True)
voronoi_point_polys = rt.isedgarVoronoi(circles, Box=_box_, use_circle_radius=False)
svg_voronoi = []
for i in range(len(voronoi_polys)):
    d = f'M {voronoi_polys[i][0][0]} {voronoi_polys[i][0][1]} '
    for j in range(1, len(voronoi_polys[i])): d += f'L {voronoi_polys[i][j][0]} {voronoi_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1"/>')
for i in range(len(voronoi_polys)):
    for j in range(len(voronoi_polys[i])):
        x, y = voronoi_polys[i][j]
        _cross_l_ = 3
        svg_voronoi.append(f'<line x1="{x-_cross_l_}" y1="{y-_cross_l_}" x2="{x+_cross_l_}" y2="{y+_cross_l_}" stroke="#000000" stroke-width="0.2"/>')
        svg_voronoi.append(f'<line x1="{x+_cross_l_}" y1="{y-_cross_l_}" x2="{x-_cross_l_}" y2="{y+_cross_l_}" stroke="#000000" stroke-width="0.2"/>')
        svg_voronoi.append(f'<circle cx="{x}" cy="{y}" r="0.5" fill="#000000"/>')
svg_voronoi_point = []
for i in range(len(voronoi_point_polys)):
    d = f'M {voronoi_point_polys[i][0][0]} {voronoi_point_polys[i][0][1]} '
    for j in range(1, len(voronoi_point_polys[i])): d += f'L {voronoi_point_polys[i][j][0]} {voronoi_point_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi_point.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1"/>')
_seen_ = set()
for i in range(len(voronoi_point_polys)):
    _color_ = rt.co_mgr.getColor(i)
    for j in range(len(voronoi_point_polys[i])):
        x, y = voronoi_point_polys[i][j]
        if (x,y) not in _seen_:
            svg_voronoi_point.append(f'<circle cx="{x}" cy="{y}" r="{5.0 + random.random()*10.0}" stroke="{_color_}" stroke-width="2.0" fill="none"/>')
        _seen_.add((x,y))

In [ ]:
# Group together "close" points from the delauney triangulation
# - and remerge them into voronoi points merged
vpoints, vpoints_pt = set(), set()
for i in range(len(voronoi_polys)):
    for j in range(len(voronoi_polys[i])):
        _xy_ = voronoi_polys[i][j]
        vpoints.add(_xy_)
    for j in range(len(voronoi_point_polys[i])):
        _xy_ = voronoi_point_polys[i][j]
        vpoints_pt.add(_xy_)
print(len(vpoints), len(vpoints_pt))

rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi_point)+''.join(svg_voronoi)+'</svg>'])

In [ ]:
# Determine which pairs need to be merged
_merge_pairs_ = set()
for a in vpoints_pt:
    for b in vpoints_pt:
        if a != b and rt.segmentLength((a,b)) < 0.1:
            _merge_pairs_.add((a,b))
pt_to_merge_pt = {}
# Do the merge...
for _pair_ in _merge_pairs_:
    a, b = _pair_
    if a not in pt_to_merge_pt:
        # Loop through all points that need to be merged
        # ... keep doing that until the merge set stabilizes
        _to_merge_          = set([a,b])
        _last_to_merge_len_ = len(_to_merge_)
        _to_merge_len_      = 0
        while _last_to_merge_len_ != _to_merge_len_:
            _last_to_merge_len_ = len(_to_merge_)
            for _pair_ in _merge_pairs_:
                a, b = _pair_
                if a in _to_merge_ or b in _to_merge_: _to_merge_ |= set([a,b])
            _to_merge_len_ = len(_to_merge_)
    # Average Point
    _x_, _y_ = 0.0, 0.0
    for _xy_ in _to_merge_:
        _x_, _y_ = _x_ + _xy_[0], _y_ + _xy_[1]
    _x_, _y_ = _x_ / len(_to_merge_), _y_ / len(_to_merge_)
    # Setup the lookup
    for _xy_ in _to_merge_: pt_to_merge_pt[_xy_] = (_x_, _y_)
# Add points that don't need to be merged
for a in vpoints_pt:
    if a not in pt_to_merge_pt: pt_to_merge_pt[a] = a
print(len(pt_to_merge_pt.keys()))